# Supplementary Figures - RNA-seq & ATAC-seq Volcano Plots

In [1]:
# set correct working directory -> project folder
getwd()
setwd('..')
getwd()

[1] "/home/sreichl/projects/bmdm-stim/workflow/notebooks"

[1] "/home/sreichl/projects/bmdm-stim"

In [2]:
# library
library(EnhancedVolcano, quietly=TRUE)
library(patchwork, quietly=TRUE)
library(ggplot2)

Registered S3 methods overwritten by 'ggalt':
  method                  from   
  grid.draw.absoluteGrob  ggplot2
  grobHeight.absoluteGrob ggplot2
  grobWidth.absoluteGrob  ggplot2
  grobX.absoluteGrob      ggplot2
  grobY.absoluteGrob      ggplot2



In [5]:
# default plotting theme
macro_theme <- function(){
    
    # settings
    font <- "Arial"
    size <- 8
    
    theme_bw(
        base_size=size,
        base_family = font
    ) %+replace% 
    
    theme(
      #grid elements
#       panel.grid.major = element_blank(),    #strip major gridlines
#       panel.grid.minor = element_blank(),    #strip minor gridlines
#       axis.ticks = element_blank(),          #strip axis ticks
      
#       strips axis lines ?
      
      #text elements
        text = element_text(              
                   family = font,           
                   size = size),
        
      plot.title = element_text(             #title
                   family = font,            #set font family
                   size = size,                #set font size
                   face = 'bold',            #bold typeface
                   hjust = 0.5,                #center align
                   vjust = 2),               #raise slightly
      
      plot.subtitle = element_text(          #subtitle
                   family = font,            #font family
                   size = size),               #font size
      
      plot.caption = element_text(           #caption
                   family = font,            #font family
                   size = size,                 #font size
                   hjust = 0.5),               #center align
      
      axis.title = element_text(             #axis titles
                   family = font,            #font family
                   size = size),               #font size
      
      axis.text = element_text(              #axis text
                   family = font,            #axis famuly
                   size = size),                #font size
        
        legend.text = element_text(              #axis text
                   family = font,            #axis famuly
                   size = size), 
        
        legend.title = element_text(              #axis text
                   family = font,            #axis famuly
                   size = size),
      
#       axis.text.x = element_text(            #margin for axis text
#                     margin=margin(5, b = 10))
    )
}

In [6]:
# function to format treatmens
treatment_format <- function(x,...){
    x <- gsub('untreated', 'Untreated', x)
    
    x <- gsub('LO28', 'Listeria', x)
    
    x <- gsub('C_albicans', 'Candida', x) 
    x <- gsub('C albicans', 'Candida', x) 
    
    x <- gsub('IFN_gamma', 'IFN-g', x)
    x <- gsub('IFN gamma', "IFN-g", x)
    
    x <- gsub('IFN_beta', 'IFN-b', x)
    x <- gsub('IFN beta', 'IFN-b', x)
    
    x <- gsub('LCMV_Cl13', 'LCMV', x)
    x <- gsub('LCMV Cl13', 'LCMV', x)
    
    x <- gsub('_', ' ', x)
    
    x <- gsub('-2h', ' 2h', x)
    x <- gsub('-6h', ' 6h', x)
    x <- gsub('-24h', ' 24h', x)
    
    return(x)
}

In [7]:
# extended ggsave
ggsave_new <- function(filename, results_path, plot, width=5, height=5){
    for (format in c('svg','png')){
        ggsave(
          paste0(filename,'.',format),
          plot = plot,
          device = format,
          path = file.path(results_path),
          scale = 1,
          dpi = 300,
            width = width,
            height = height,
          limitsize = FALSE,
        )
    }
}

# configs

In [8]:
# configs
data_RNA <- file.path('results','RNA')
data_ATAC <- file.path('results','ATAC','all')

results_path <- file.path('results','figures','supp_bulk')

In [9]:
# make directories if not exist
dir.create(results_path, showWarnings = FALSE, recursive = TRUE)

# Load annotations

In [10]:
# load RNA annotations
RNA_annot <- read.csv(file=file.path('metadata','RNA_sample_metadata.csv'), row.names=1)
dim(RNA_annot)
head(RNA_annot)

[1] 66 10

,library,timepoint.after.thawing,cell_line,Treatment,Treatment_time,mouse_pool,sex,organism,bio.replicate,experiment_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
RNA_PT76_R1_C_albicans_2h,Quant-seq,d010,BMDM,C_albicans,2h,PT62_c,female,mouse,R1,PT76
RNA_PT82_R1_C_albicans_24h,Quant-seq,d010,BMDM,C_albicans,24h,PT62_c,female,mouse,R1,PT82
RNA_PT76_R1_C_albicans_4h,Quant-seq,d010,BMDM,C_albicans,4h,PT62_c,female,mouse,R1,PT76
RNA_PT76_R1_C_albicans_6h,Quant-seq,d010,BMDM,C_albicans,6h,PT62_c,female,mouse,R1,PT76
RNA_PT82_R1_C_albicans_8h,Quant-seq,d010,BMDM,C_albicans,8h,PT62_c,female,mouse,R1,PT82
RNA_PT76_R1_untreated_0h,Quant-seq,d010,BMDM,untreated,0h,PT62_c,female,mouse,R1,PT76


In [11]:
RNA_annot[RNA_annot$Treatment=='untreated','Treatment_time'] <- '0h'

In [12]:
# load ATAC annotations
ATAC_annot <- read.csv(file=file.path('metadata','ATAC_sample_metadata.csv'), row.names=1)
dim(ATAC_annot)
head(ATAC_annot)

[1] 78 10

,library,timepoint.after.thawing,cell_line,Treatment,Treatment_time,mouse_pool,sex,bio.replicate,organism,experiment_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
PT76_R1_C_albicans_2h,ATAC-seq,d010,BMDM,C_albicans,2h,PT62_c,female,R1,mouse,PT76
PT76_R1_C_albicans_4h,ATAC-seq,d010,BMDM,C_albicans,4h,PT62_c,female,R1,mouse,PT76
PT76_R1_C_albicans_6h,ATAC-seq,d010,BMDM,C_albicans,6h,PT62_c,female,R1,mouse,PT76
PT76_R1_C_albicans_8h,ATAC-seq,d010,BMDM,C_albicans,8h,PT62_c,female,R1,mouse,PT76
PT76_R1_untreated_0h,ATAC-seq,d010,BMDM,untreated,0h,PT62_c,female,R1,mouse,PT76
PT76_R1_IFN_beta_2h,ATAC-seq,d010,BMDM,IFN_beta,2h,PT62_c,female,R1,mouse,PT76


In [13]:
ATAC_annot[ATAC_annot$Treatment=='untreated','Treatment_time'] <- '0h'

# Volcano Panel RNA DEA results

In [14]:
pCutoff <- 0.05
FCcutoff <- 2

In [15]:
# plot specifications
n_col <- 5
width <- 3
height <- 3
width_panel <- n_col * width
height_panel <- height * 25/n_col

In [18]:
volcano_plots <- list()
for (treatment in unique(RNA_annot$Treatment)){
    if (treatment=='untreated'){
        next
    }
    
    tmp_dea <- read.csv(file=file.path(data_RNA,'DEA',paste0('DEA_',treatment,'.tsv')), sep='\t')

    for (group in (sort(unique(tmp_dea$group))[c(2,3,4,5,1)])){
        toptable <- tmp_dea[tmp_dea$group==group,]
        lab <- '' #toptable$rn
        x <- "logFC"
        y <- "adj.P.Val"

        volcano_plots[[group]] <- EnhancedVolcano(toptable = toptable,
                        lab = lab,
                        x = x,
                        y = y,
                        selectLab = NULL,
                        xlim = c(min(toptable[[x]], na.rm = TRUE) - 1, max(toptable[[x]], na.rm = TRUE) + 1),
                        ylim = c(0, max(-log10(toptable[[y]]), na.rm = TRUE) + 5),
                        xlab = bquote(~log[2] ~ "fold change"),
                        ylab = bquote(~-log[10] ~ "adjusted p-value"),
                        axisLabSize = 12,
                        title = treatment_format(group),
                        subtitle = '',
                        caption = paste0("variables:",nrow(toptable),"; log2FC>",FCcutoff,"; adj.p-val<",pCutoff),
                        titleLabSize = 14,
                        subtitleLabSize = 0,
                        captionLabSize = 6,
                        pCutoff = pCutoff, #default: 0.05
                        pCutoffCol = y,
                        FCcutoff = FCcutoff, # default:1
                        cutoffLineType = "longdash",
                        cutoffLineCol = "black",
                        cutoffLineWidth = 0.4,
                        pointSize = 0.25, # default: 2
                        labSize = 3, #default: 5
                        labCol = "black",
                        labFace = "plain",
                        boxedLabels = TRUE, #default: FALSE
                        parseLabels = FALSE,
                        shape = 19,
                        shapeCustom = NULL,
                        col = c("grey30", "forestgreen", "royalblue", "red2"),
                        colCustom = NULL,
                        colAlpha = 1/2,
                        colGradient = NULL,
                        colGradientBreaks = c(pCutoff, 1),
                        colGradientLabels = c("0", "1.0"),
                        colGradientLimits = c(0, 1),
                        legendLabels = c("NS", expression(log[2] ~ FC), "adj. p-value", 'both'),
                        legendPosition = "right", #default: "top"
                        legendLabSize = 14,
                        legendIconSize = 4,
                        legendDropLevels = TRUE,
                        encircle = NULL,
                        encircleCol = "black",
                        encircleFill = "pink",
                        encircleAlpha = 3/4,
                        encircleSize = 2.5,
                        shade = NULL,
                        shadeFill = "grey",
                        shadeAlpha = 1/2,
                        shadeSize = 0.01,
                        shadeBins = 2,
                        drawConnectors = TRUE, #default: FALSE
                        widthConnectors = 0.1, # default: 0.5
                        typeConnectors = "closed",
                        endsConnectors = "first",
                        lengthConnectors = unit(0.01, "npc"),
                        colConnectors = "grey10",
                        max.overlaps = 0,
                        maxoverlapsConnectors = 10, # default: NULL
                        min.segment.length = 0,
                        directionConnectors = "both",
                        arrowheads = FALSE, # default: TRUE
                        hline = NULL,
                        hlineType = "longdash",
                        hlineCol = "black",
                        hlineWidth = 0.4,
                        vline = NULL,
                        vlineType = "longdash",
                        vlineCol = "black",
                        vlineWidth = 0.4,
                        gridlines.major = TRUE,
                        gridlines.minor = TRUE,
                        border = "partial",
                        borderWidth = 0.8,
                        borderColour = "black",
                        raster = FALSE
                       ) + macro_theme() + theme(legend.title = element_blank())

    }
}

In [19]:
volcano_plots_panel <- wrap_plots(volcano_plots, ncol = n_col, guides = "collect")

In [20]:
# save plot
options(repr.plot.width=width_panel, repr.plot.height=height_panel)
# print(volcano_plots_panel)

ggsave_new(filename = "RNA_DEA_volcanos", 
           results_path=results_path, 
           plot=volcano_plots_panel, 
           width=width_panel, 
           height=height_panel)

# Volcano Panel ATAC DEA results

In [21]:
pCutoff <- 0.05
FCcutoff <- 2

In [22]:
# plot specifications
n_col <- 5
width <- 3
height <- 3
width_panel <- n_col * width
height_panel <- height * 25/n_col

In [24]:
volcano_plots <- list()
for (treatment in unique(ATAC_annot$Treatment)){
    if (treatment=='untreated'){
        next
    }
    
    tmp_dea <- read.csv(file=file.path(data_ATAC,'DEA',paste0('DEA_',treatment,'.tsv')), sep='\t')

    for (group in (sort(unique(tmp_dea$group))[c(2,3,4,5,1)])){
        toptable <- tmp_dea[tmp_dea$group==group,]
        lab <- '' #toptable$rn
        x <- "logFC"
        y <- "adj.P.Val"

        volcano_plots[[group]] <- EnhancedVolcano(toptable = toptable,
                        lab = lab,
                        x = x,
                        y = y,
                        selectLab = NULL,
                        xlim = c(min(toptable[[x]], na.rm = TRUE) - 1, max(toptable[[x]], na.rm = TRUE) + 1),
                        ylim = c(0, max(-log10(toptable[[y]]), na.rm = TRUE) + 5),
                        xlab = bquote(~log[2] ~ "fold change"),
                        ylab = bquote(~-log[10] ~ "adjusted p-value"),
                        axisLabSize = 12,
                        title = treatment_format(group),
                        subtitle = '',
                        caption = paste0("variables:",nrow(toptable),"; log2FC>",FCcutoff,"; adj.p-val<",pCutoff),
                        titleLabSize = 14,
                        subtitleLabSize = 0,
                        captionLabSize = 6,
                        pCutoff = pCutoff, #default: 0.05
                        pCutoffCol = y,
                        FCcutoff = FCcutoff, # default:1
                        cutoffLineType = "longdash",
                        cutoffLineCol = "black",
                        cutoffLineWidth = 0.4,
                        pointSize = 0.1, # default: 2
                        labSize = 3, #default: 5
                        labCol = "black",
                        labFace = "plain",
                        boxedLabels = TRUE, #default: FALSE
                        parseLabels = FALSE,
                        shape = 19,
                        shapeCustom = NULL,
                        col = c("grey30", "forestgreen", "royalblue", "red2"),
                        colCustom = NULL,
                        colAlpha = 1/2,
                        colGradient = NULL,
                        colGradientBreaks = c(pCutoff, 1),
                        colGradientLabels = c("0", "1.0"),
                        colGradientLimits = c(0, 1),
                        legendLabels = c("NS", expression(log[2] ~ FC), "adj. p-value", 'both'),
                        legendPosition = "right", #default: "top"
                        legendLabSize = 14,
                        legendIconSize = 4,
                        legendDropLevels = TRUE,
                        encircle = NULL,
                        encircleCol = "black",
                        encircleFill = "pink",
                        encircleAlpha = 3/4,
                        encircleSize = 2.5,
                        shade = NULL,
                        shadeFill = "grey",
                        shadeAlpha = 1/2,
                        shadeSize = 0.01,
                        shadeBins = 2,
                        drawConnectors = TRUE, #default: FALSE
                        widthConnectors = 0.1, # default: 0.5
                        typeConnectors = "closed",
                        endsConnectors = "first",
                        lengthConnectors = unit(0.01, "npc"),
                        colConnectors = "grey10",
                        max.overlaps = 0,
                        maxoverlapsConnectors = 10, # default: NULL
                        min.segment.length = 0,
                        directionConnectors = "both",
                        arrowheads = FALSE, # default: TRUE
                        hline = NULL,
                        hlineType = "longdash",
                        hlineCol = "black",
                        hlineWidth = 0.4,
                        vline = NULL,
                        vlineType = "longdash",
                        vlineCol = "black",
                        vlineWidth = 0.4,
                        gridlines.major = TRUE,
                        gridlines.minor = TRUE,
                        border = "partial",
                        borderWidth = 0.8,
                        borderColour = "black",
                        raster = FALSE
                       ) + macro_theme() + theme(legend.title = element_blank())

    }
}

In [25]:
volcano_plots_panel <- wrap_plots(volcano_plots, ncol = n_col, guides = "collect")

In [ ]:
# save plot

# only png, bc too many points (svg > 500MB)
format <- 'png'
ggsave(
          paste0("ATAC_DEA_volcanos",'.',format),
          plot = volcano_plots_panel,
          device = format,
          path = file.path(results_path),
          scale = 1,
          dpi = 300,
            width = width_panel,
            height = height_panel,
          limitsize = FALSE,
        )